# INDIC-BERT

In [1]:
!pip3 install transformers
!pip3 install datasets
!pip3 install sentencepiece
!pip3 install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=6d5291df526cadd44ea3313cf971daffd5c6c9f9d865b5256d741168175874fd
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


### LOADING DATASET

In [2]:
from datasets import ClassLabel, load_dataset, load_metric, DownloadMode

lang='hi'

raw_datasets = load_dataset('ai4bharat/naamapadam', lang)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset naamapadam_pr downloaded and prepared to /root/.cache/huggingface/datasets/ai4bharat___naamapadam_pr/hi/1.0.0/99b5ec77eabfaa3fbff510d8cf70d7c34519486cb7dbee99ede19474ddff9b20. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
column_names = raw_datasets["train"].column_names
features = raw_datasets["train"].features
print(column_names)
print(features)
text_column_name = "tokens"
label_column_name = "ner_tags"

['tokens', 'ner_tags']
{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'ner_tags': Sequence(feature=ClassLabel(num_classes=7, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)}


In [4]:
label_list = features[label_column_name].feature.names

label_to_id = {label_list[i]: features[label_column_name].feature.str2int( label_list[i] ) for i in range(len(label_list))}

print(label_to_id)

num_labels = len(label_list)

{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6}


In [5]:
from transformers import AutoModelForTokenClassification, AutoConfig, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification, EarlyStoppingCallback, IntervalStrategy
import numpy as np

config = AutoConfig.from_pretrained('ai4bharat/indic-bert', num_labels=num_labels, finetuning_task='ner')
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
model = AutoModelForTokenClassification.from_pretrained('ai4bharat/indic-bert', num_labels=num_labels )

2024-03-13 06:49:44.001048: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-13 06:49:44.001162: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-13 06:49:44.121895: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/5.65M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/135M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of AlbertForTokenClassification were not initialized from the model checkpoint at ai4bharat/indic-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# Tokenize all texts and align the labels with them.
padding = "max_length"
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples[text_column_name],
        padding=padding,
        truncation=True,
        max_length=512,
        # We use this argument because the texts in our dataset are lists of words (with a label for each word).
        is_split_into_words=True,
    )
    labels = []
    for i, label in enumerate(examples[label_column_name]):
        # print('=====')
        # print('{} {}'.format(i,label)) #ak
        word_ids = tokenized_inputs.word_ids(batch_index=i)

        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

### LOADING TRAIN SET AND VALIDATION SET

In [7]:
train_datasett = raw_datasets["train"]
train_dataset=train_datasett.select(range(20000))
train_dataset = train_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on train dataset",
)

Running tokenizer on train dataset #0:   0%|          | 0/5 [00:00<?, ?ba/s]

Running tokenizer on train dataset #2:   0%|          | 0/5 [00:00<?, ?ba/s]

Running tokenizer on train dataset #1:   0%|          | 0/5 [00:00<?, ?ba/s]

Running tokenizer on train dataset #3:   0%|          | 0/5 [00:00<?, ?ba/s]

In [8]:
eval_dataset = raw_datasets["validation"]
eval_dataset = eval_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on Validation dataset",
)

Running tokenizer on Validation dataset #0:   0%|          | 0/4 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #1:   0%|          | 0/4 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #2:   0%|          | 0/4 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #3:   0%|          | 0/4 [00:00<?, ?ba/s]

In [9]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [10]:
# Metrics
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    # Unpack nested dictionaries
    final_results = {}
    for key, value in results.items():
        if isinstance(value, dict):
            for n, v in value.items():
                final_results[f"{key}_{n}"] = v
        else:
            final_results[key] = value
    return final_results

### SETTING TRAINING ARGUMENTS

In [47]:
# args=TrainingArguments(output_dir='output_dir',max_steps=5)
batch_size=10
args=TrainingArguments(
    output_dir='output_dir',
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    evaluation_strategy = "epoch",
    learning_rate=5e-7)

In [48]:
# Initialize our Trainer
# early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=2)
# args.metric_for_best_model = "f1"
# args.load_best_model_at_end = True
# args.evaluation_strategy = IntervalStrategy.STEPS
# args.eval_steps = args.save_steps
# args.greater_is_better = True

trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    # callbacks=[early_stopping_callback],
    args=args,
)

In [49]:
trainer.args

TrainingArguments(
_n_gpu=2,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_v2': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_

### TRAINING THE MODEL

In [50]:
train_result = trainer.train()
metrics = train_result.metrics

Epoch,Training Loss,Validation Loss,Loc Precision,Loc Recall,Loc F1,Loc Number,Org Precision,Org Recall,Org F1,Org Number,Per Precision,Per Recall,Per F1,Per Number,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
1,0.171900,0.261670,0.717105,0.736414,0.726632,10213,0.581300,0.552728,0.566654,9786,0.714773,0.713285,0.714029,10568,0.674643,0.669611,0.672118,0.921297
2,0.170500,0.262180,0.715469,0.735925,0.725553,10213,0.577569,0.552013,0.564502,9786,0.714651,0.713096,0.713873,10568,0.672763,0.669153,0.670953,0.921067
3,0.176300,0.262522,0.717559,0.735827,0.726578,10213,0.576587,0.553137,0.564619,9786,0.712883,0.714232,0.713556,10568,0.672469,0.669873,0.671168,0.921047


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

In [51]:
metrics = trainer.evaluate()

trainer.log_metrics("eval", metrics)

***** eval metrics *****
  epoch                   =        3.0
  eval_LOC_f1             =     0.7266
  eval_LOC_number         =      10213
  eval_LOC_precision      =     0.7176
  eval_LOC_recall         =     0.7358
  eval_ORG_f1             =     0.5646
  eval_ORG_number         =       9786
  eval_ORG_precision      =     0.5766
  eval_ORG_recall         =     0.5531
  eval_PER_f1             =     0.7136
  eval_PER_number         =      10568
  eval_PER_precision      =     0.7129
  eval_PER_recall         =     0.7142
  eval_loss               =     0.2625
  eval_overall_accuracy   =      0.921
  eval_overall_f1         =     0.6712
  eval_overall_precision  =     0.6725
  eval_overall_recall     =     0.6699
  eval_runtime            = 0:04:15.04
  eval_samples_per_second =     52.775
  eval_steps_per_second   =      2.639


### TOKENIZING TEST SET FOR PREDICTION

In [54]:
tokenized_test_set = {}

tokenized_test_set = raw_datasets['test'].map(
  tokenize_and_align_labels,
  batched=True,
  num_proc=4,
  load_from_cache_file=True,
  desc="Running tokenizer on test dataset of language {0}".format('hi'),
)

Running tokenizer on test dataset of language hi #0:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset of language hi #1:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset of language hi #2:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset of language hi #3:   0%|          | 0/1 [00:00<?, ?ba/s]

### RUNNING TEST SET PREDICTION

In [55]:
final_metrics = {}


predictions, labels, metrics = trainer.predict(tokenized_test_set, metric_key_prefix='hi')

lang_specific_results = {}
for key in metrics:
  if 'overall_precision' in key:
    lang_specific_results['Precision'] = metrics[key]
  elif 'overall_recall' in key:
    lang_specific_results['Recall'] = metrics[key]
  elif 'overall_f1' in key:
    lang_specific_results['F1'] = metrics[key]
final_metrics = lang_specific_results


In [56]:
print("Test Set: ",final_metrics)

Test Set:  {'Precision': 0.6848760748609004, 'Recall': 0.7019180922757906, 'F1': 0.6932923707117256}


In [52]:
model.save_pretrained("BERT_model")

In [57]:
## Save tokenizer
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [24]:
# !zip -r BertTuned.zip /kaggle/working/ner_model

In [53]:
model = AutoModelForTokenClassification.from_pretrained('/kaggle/working/BERT_model')

### CONFIGURING THE MODEL FOR PREDICTION

In [58]:
id2label = {
    str(i): label for i,label in enumerate(label_list)
}
label2id = {
    label: str(i) for i,label in enumerate(label_list)
}
print(id2label)
print(label2id)

{'0': 'O', '1': 'B-PER', '2': 'I-PER', '3': 'B-ORG', '4': 'I-ORG', '5': 'B-LOC', '6': 'I-LOC'}
{'O': '0', 'B-PER': '1', 'I-PER': '2', 'B-ORG': '3', 'I-ORG': '4', 'B-LOC': '5', 'I-LOC': '6'}


In [59]:
import json
config=json.load(open("/kaggle/working/BERT_model/config.json"))

In [60]:
config["id2label"] = id2label
config["label2id"] = label2id

In [61]:
json.dump(config, open("/kaggle/working/BERT_model/config.json","w"))

In [62]:
model = AutoModelForTokenClassification.from_pretrained("BERT_model")

### MAKING OBJECT OF OUR SENTENCES TO DO PREDICTION

In [63]:
from datasets import Dataset
data={
"tokens": [["पूरी","दुनिया","में","भूगोल","पर","पिछली","कई","सदियों","से","शोध","होते","आ","रहे","हैं","और","आने","वाले","समय","में","भी","भूगोल","धरती","को","जानने","समझने","का","सर्वश्रेष्ठ","माध्यम","बना","रहेगा","।"],
   ["भारत","में","इस","टीवी","की","कीमत","8,00,000","रुपये","रखी","गयी","है","।"],
   ["टेक्सास","में","निजी","यात्री","विमान","दुर्घटनाग्रस्त",",","दस","लोगों","की","मौत"],
   ["ये","कुछ","वाक्य","प्रयोग","हमने","अपनी","हिंदी","की","जानकारी","देखने","के","लिये","किये","।"],
   ["राष्ट्रीय","सुरक्षा","से","जुड़े","मुद्दों","पर","सरकार","के","साथ","एकजुट","होने","के","लिए","विपक्ष","से","अपील","करते","हुए",",","।"],
   ["उन्होंने","कहा","कि","प्रदेश","सरकार","जनसामान्य","को","आवश्यक","वस्तुएं","उचित","मूल्य","एवं","निर्धारित","मात्रा","में","उपलब्ध","कराने","के","लिए","प्रतिबद्घ","है","और","इसके","लिए","समय","-","समय","पर","निर्देश","जारी","किए","जाते","हैं","बावजूद","इसके","हाल","के","दिनों","में","कई","जिंसों","की","कीमतों","में","तेजी","इजाफा","हुआ","है","।"],
   ["इसके","बाद","श्रीलंका","के","खिलाफ","264","रन","तब","बनाए","थे",",","जब","मैं","उंगली","में","चोट","लगने","की","वजह","से","तीन","महीने","बाद","वापसी","करर","रहा","था","और","नर्वस","था","कि","मैं","रन","बना","पाऊंगा","या","नहीं","और","अब","208*","तब","बनाए",",","जब","सीरीज","का","पहला","मैच","गंवाने","के","बाद","दूसरे","मैच","में","वापसी","करना","जरूरी","था","।"],
    ["भारत","का","मीडिया","टीवी","चैनल","या","अखबार","भारत-इज़राइल","रििश्ते","पर","विस्तृत","जानकारी","नहीं","देते","हैं","लेकिन","इज़राइल","की","अखबारों","से","पता","चलता","है","कि","भारत-इज़राइल","रिश्ता","पिछले","कुछ","सालों","में","कितना","प्रगाढ़","और","गहरा","हो","गया","है","."],
    ["सुनवाई","के","दौरान","कार्यकर्ता","ने","कहा","कि","उन्हें","अपनी","जिंदगी","से","प्यार","है","और","वह","अफ्सपा","हटाने","की","मांग","को","लेकर","अनशन","को","हथियार","के","तौर","पर","इस्तेमाल","कर","रही","है","क्योंकि","इसका","‘ज्यादा","असर","होगा","।","उनका","कहना","है","कि","यह","अपराध","नहीं","है","."],
   ["दोनों","प्रधानमंत्रियों","ने","सिविल","न्यूक्लियर","एनर्जी","के","क्षेत्र","में","खासकर","तकनीकी","प्रशिक्षण","की","दिशा","काम","करने","पर","सहमति","जताई","है","."],
   ["दोनों","बदमााश","शामली","के","लोहारी","जलालाबाद","के","रहने","वाले","थे","और","दोनों","पर","50-50","हजार","रुपए","का","इनाम","था","।"],
   ["बाद","में","दोननों","समुदाय","की","तरफ","से","हिंसा","भड़क","उठी","।"],
   ["30","फीसदी","से","कम","अंक","लाने","वालों","को","असफल","घोषित","कर","दिया","जाएगा","."],
    ["ट्रेलर","में","दर्शाया","गया","है","कि","कोई","इसे","मारने","के","लिए","सात","दिनों","का","समय","तय","करता","है","।"],
    ["ऐसे","में","लोगों","को","इलाज","के","लिये","बल्लभगढ़","व","फरीदाबाद","जाना","पड़ता","है","।"],
   ["आल्पस","कंपनी","के","बाहर","धरने","पर","बैठे","14०","कर्मचारियों","ने","अपनी","मांगों","को","लेकर","जमकर","नारेबाजी","कर","धरना","प्रदर्शन","किया","।"],
    ["बजाज","के","सवाल","को","सुनने","के","बादद","मुझे","यकीन","है","कि","कोई","भी","इस","दावे","को","नहीं","मानेगा","कि","लोग","डरे","हुए","हैं","।"],
    ["फिर","हीरो-हीरोइन","हमेशा","तो","साथ","रह","नहीं","सकते","तो","एकविरह","गीत","तो","गाना","ही","पड़ेगा","हीरो","को","(","याद","आ","रही","है","टाइप",")","।"],
    ["जानकारी","के","अनुसार","क्षेत्रीय","अस्पताल","मंे","इस","समय","विशेषज्ञ","चिकित्सकों","के","25","पद","सृजित","किए","गए","हैं","जिनमें","से","17","पद","हही","भरे","हैं",",","एक","डाक्टर","आईजीएमसी","मंे","डेपुटेशन","पर","है",",","जबकि","शेष","खाली","चल","रहे","हैं","।"],
    ["साइबर","क्षेत्र","मे","घुसपैठ","करके","हमारे","संचार","व","सूचना","तंत्र","में","सेंध","लगा","रहा","है","।"],
    ["हरियाणा","के","वित्त","एवं","राजस्व","मंत्री","ककैप्टन","अभिमन्यु","ने","कहा","कि","खाप","व्यवस्था","भारत","की","मूल","शासन","व्यवस्था","है","पढ़ें"],
    ["अपनी","इस","हालत","के","लिए","महिला","कैदी","ने","जेल","प्रबंधन","को","जिम्मेदार","ठहराया","."],
    ["यह","तो","जगज़ाहिर","है","कि","खेल","और","विवाद","दोनों","का","आपस","में","चोली","-दामन","का","साथ","है","."],
   ["इसके","बादवाले","एक","माह","तक","कुछ","पका","हुआ","आहार","दिया","जा","सकता","है","।"],
   ["हालांकि","रक्षा","मंत्रालय","में","उनका","कार्यकाल","आसान","नहीं","था","क्योंकि","वह","एक","ऐसे","नेता","के","रूप","में","सफल","रहीं","जिन्होंने","उद्योग","और","सशस्त्र","बलों","दोनों","का","विश्वास","जीता","था","."]],

"ner_tags": [[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [5,0,0,0,0,0,0,0,0,0,0,0],
    [5,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,5,0,5,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,6,0,6,0,0,0,0],
    [3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,5,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [5,0,0,0,0,0,0,1,0,0,0,0,0,5,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]
}

dataset=Dataset.from_dict(data)
print(dataset)


Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 25
})


### TOKENISING OUR SENTENCES

In [64]:
tokenized_test_set = {}

tokenized_test_set = dataset.map(
  tokenize_and_align_labels,
  batched=True,
  num_proc=4,
  load_from_cache_file=True,
  desc="Running tokenizer on test dataset of language {0}".format('hi'),
)

Running tokenizer on test dataset of language hi #0:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset of language hi #1:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset of language hi #3:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset of language hi #2:   0%|          | 0/1 [00:00<?, ?ba/s]

### RUNNING PREDICTION ON OUR SENTENCES

In [65]:
final_metrics = {}

predictions, labels, metrics = trainer.predict(tokenized_test_set, metric_key_prefix='hi')

lang_specific_results = {}
for key in metrics:
  if 'overall_precision' in key:
    lang_specific_results['Precision'] = metrics[key]
  elif 'overall_recall' in key:
    lang_specific_results['Recall'] = metrics[key]
  elif 'overall_f1' in key:
    lang_specific_results['F1'] = metrics[key]
final_metrics = lang_specific_results

print(final_metrics)

{'Precision': 0.2916666666666667, 'Recall': 0.4666666666666667, 'F1': 0.35897435897435903}


In [66]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

### LOADING ALL SENTENCES

In [67]:
sentences=['पूरी दुनिया में भूगोल पर पिछली कई सदियों से शोध होते आ रहे हैं और आने वाले समय में भी भूगोल धरती को जानने समझने का सर्वश्रेष्ठ माध्यम बना रहेगा।',
       'भारत में इस टीवी की कीमत 8,00,000 रुपये रखी गयी है।',
       'टेक्सास में निजी यात्री विमान दुर्घटनाग्रस्त, दस लोगों की मौत',
       'ये कुछ वाक्य प्रयोग हमने अपनी हिंदी की जानकारी देखने के लिये किये।',
       'राष्ट्रीय सुरक्षा से जुड़े मुद्दों पर सरकार के साथ एकजुट होने के लिए विपक्ष से अपील करते हुए, ।',
       'उन्होंने कहा कि प्रदेश सरकार जनसामान्य को आवश्यक वस्तुएं उचित मूल्य एवं निर्धारित मात्रा में उपलब्ध कराने के लिए प्रतिबद्घ है और इसके लिए समय-समय पर निर्देश जारी किए जाते हैं, बावजूद इसके हाल के दिनों में कई जिंसों की कीमतों में तेजी इजाफा हुआ है।',
       'इसके बाद श्रीलंका के खिलाफ 264 रन तब बनाए थे, जब मैं उंगली में चोट लगने की वजह से तीन महीने बाद वापसी कर रहा था और नर्वस था कि मैं रन बना पाऊंगा या नहीं और अब 208* तब बनाए, जब सीरीज का पहला मैच गंवाने के बाद दूसरे मैच में वापसी करना जरूरी था।',
       'भारत का मीडिया, टीवी चैनल या अखबार, भारत-इज़राइल रिश्ते पर विस्तृत जानकारी नहीं देते हैं, लेकिन इज़राइल की अखबारों से पता चलता है कि भारत-इज़राइल रिश्ता पिछले कुछ सालों में कितना प्रगाढ़ और गहरा हो गया है.',
       'सुनवाई के दौरान कार्यकर्ता ने कहा कि उन्हें अपनी जिंदगी से प्यार है और वह अफ्सपा हटाने की मांग को लेकर अनशन को हथियार के तौर पर इस्तेमाल कर रही हैं क्योंकि इसका ‘ज्यादा असर’ होगा. उनका कहना है कि यह अपराध नहीं है.',
        'दोनों प्रधानमंत्रियों ने सिविल न्यूक्लियर एनर्जी के क्षेत्र में खासकर तकनीकी प्रशिक्षण की दिशा काम करने पर सहमति जताई है.',
        'दोनों बदमाश शामली के लोहारी जलालाबाद के रहने वाले थे और दोनों पर 50-50 हजार रुपए का इनाम था।',
        'बाद में दोनों समुदाय की तरफ से हिंसा भड़क उठी।',
        '30 फीसदी से कम अंक लाने वालों को असफल घोषित कर दिया जाएगा.',
        'ट्रेलर में दर्शाया गया है कि कोई इसे मारने के लिए सात दिनों का समय तय करता है।',
        'ऐसे में लोगों को इलाज के लिये बल्लभगढ़ व फरीदाबाद जाना पड़ता है।',
        'आल्पस कंपनी के बाहर धरने पर बैठे 14० कर्मचारियों ने अपनी मांगों को लेकर जमकर नारेबाजी कर धरना प्रदर्शन किया।',
        'बजाज के सवाल को सुनने के बाद मुझे यकीन है कि कोई भी इस दावे को नहीं मानेगा कि लोग डरे हुए हैं।',
        'फिर हीरो-हीरोइन हमेशा तो साथ रह नहीं सकते तो एक विरह गीत तो गाना ही पड़ेगा हीरो को (याद आ रही है टाइप)। ',
        'जानकारी के अनुसार क्षेत्रीय अस्पताल मंे इस समय विशेषज्ञ चिकित्सकों के 25 पद सृजित किए गए हैं, जिनमें से 17 पद ही भरे हैं, एक डाक्टर आईजीएमसी मंे डेपुटेशन पर है, जबकि शेष खाली चल रहे हैं।',
        'साइबर क्षेत्र में घुसपैठ करके हमारे संचार व सूचना तंत्र में सेंध लगा रहा है।',
        'हरियाणा के वित्त एवं राजस्व मंत्री कैप्टन अभिमन्यु ने कहा कि खाप व्यवस्था भारत की मूल शासन व्यवस्था है... पढ़ें',
        'अपनी इस हालत के लिए महिला कैदी ने जेल प्रबंधन को जिम्मेदार ठहराया.',
        'यह तो जगज़ाहिर है कि खेल और विवाद दोनों का आपस में चोली-दामन का साथ है.',
        'इसके बादवाले एक माह तक कुछ पका हुआ आहार दिया जा सकता है।',
        'हालांकि, रक्षा मंत्रालय में उनका कार्यकाल आसान नहीं था, क्योंकि वह एक ऐसे नेता के रूप में सफल रहीं, जिन्होंने उद्योग और सशस्त्र बलों दोनों का विश्वास जीता था.']


In [68]:
def get_ner(sentence):
    tok_sentence = tokenizer(sentence, return_tensors='pt')

    with torch.no_grad():
        logits = model(**tok_sentence).logits.argmax(-1)
        predicted_tokens_classes = [
            model.config.id2label[t.item()] for t in logits[0]]

        predicted_labels = []

        previous_token_id = 0
        word_ids = tok_sentence.word_ids()
        for word_index in range(len(word_ids)):
            if word_ids[word_index] == None:
                previous_token_id = word_ids[word_index]
            elif word_ids[word_index] == previous_token_id:
                previous_token_id = word_ids[word_index]
            else:
                predicted_labels.append(predicted_tokens_classes[word_index])
                previous_token_id = word_ids[word_index]

        ner_output = []
        for index in range(len(sentence.split(' '))):
            if(index<len(predicted_labels)):
                ner_output.append(sentence.split(' ')[index]+':'+predicted_labels[index])
            else:
                ner_output.append(sentence.split(' ')[index]+':'+'O')
        return ner_output

### GETTING ALL NER TAGS FOR OUR SENTENCES

In [69]:
pred_outputs=[]
for sent in sentences:
    predicted_labels = get_ner(sent)
    pred_outputs.append(predicted_labels)
for i in pred_outputs:
    print(i)

['पूरी:O', 'दुनिया:O', 'में:O', 'भूगोल:O', 'पर:O', 'पिछली:O', 'कई:O', 'सदियों:O', 'से:O', 'शोध:O', 'होते:O', 'आ:O', 'रहे:O', 'हैं:O', 'और:O', 'आने:O', 'वाले:O', 'समय:O', 'में:O', 'भी:O', 'भूगोल:O', 'धरती:B-LOC', 'को:O', 'जानने:O', 'समझने:O', 'का:O', 'सर्वश्रेष्ठ:O', 'माध्यम:O', 'बना:O', 'रहेगा।:O']
['भारत:B-LOC', 'में:O', 'इस:O', 'टीवी:O', 'की:O', 'कीमत:O', '8,00,000:O', 'रुपये:O', 'रखी:O', 'गयी:O', 'है।:O']
['टेक्सास:B-ORG', 'में:O', 'निजी:O', 'यात्री:O', 'विमान:O', 'दुर्घटनाग्रस्त,:O', 'दस:O', 'लोगों:O', 'की:O', 'मौत:O']
['ये:O', 'कुछ:O', 'वाक्य:O', 'प्रयोग:O', 'हमने:O', 'अपनी:O', 'हिंदी:O', 'की:O', 'जानकारी:O', 'देखने:O', 'के:O', 'लिये:O', 'किये।:O']
['राष्ट्रीय:O', 'सुरक्षा:O', 'से:O', 'जुड़े:O', 'मुद्दों:O', 'पर:O', 'सरकार:O', 'के:O', 'साथ:O', 'एकजुट:O', 'होने:O', 'के:O', 'लिए:O', 'विपक्ष:O', 'से:O', 'अपील:O', 'करते:O', 'हुए,:O', '।:O']
['उन्होंने:O', 'कहा:O', 'कि:O', 'प्रदेश:B-ORG', 'सरकार:O', 'जनसामान्य:O', 'को:O', 'आवश्यक:O', 'वस्तुएं:O', 'उचित:O', 'मूल्य:O', 'एवं:O', 'निर्धारि

### GROUND TRUTH FOR COMPARISON

In [70]:
Q1_List=[
    ['पूरी:O', 'दुनिया:O', 'में:O', 'भूगोल:B-MISC', 'पर:O', 'पिछली:O', 'कई:O', 'सदियों:O', 'से:O', 'शोध:O', 'होते:O', 'आ:O', 'रहे:O', 'हैं:O', 'और:O', 'आने:O', 'वाले:O', 'समय:O', 'में:O', 'भी:O', 'भूगोल:B-MISC', 'धरती:O', 'को:O', 'जानने:O', 'समझने:O', 'का:O', 'सर्वश्रेष्ठ:O', 'माध्यम:O', 'बना:O', 'रहेगा:O', '।:O'],
    ['भारत:B-LOC', 'में:O', 'इस:O', 'टीवी:O', 'की:O', 'कीमत:O', '8,00,000:B-MISC', 'रुपये:O', 'रखी:O', 'गयी:O', 'है:O', '।:O'],
    ['टेक्सास:B-LOC', 'में:O', 'निजी:O', 'यात्री:O', 'विमान:O', 'दुर्घटनाग्रस्त:O', ',:O', 'दस:O', 'लोगों:O', 'की:O', 'मौत:O'],
    ['ये:O', 'कुछ:O', 'वाक्य:O', 'प्रयोग:O', 'हमने:O', 'अपनी:O', 'हिंदी:B-MISC', 'की:O', 'जानकारी:O', 'देखने:O', 'के:O', 'लिये:O', 'किये:O', '।:O'],
    ['राष्ट्रीय:O', 'सुरक्षा:O', 'से:O', 'जुड़े:O', 'मुद्दों:O', 'पर:O', 'सरकार:O', 'के:O', 'साथ:O', 'एकजुट:O', 'होने:O', 'के:O', 'लिए:O', 'विपक्ष:B-ORG', 'से:O', 'अपील:O', 'करते:O', 'हुए:O', ',:O', '।:O'],
    ['उन्होंने:O', 'कहा:O', 'कि:O', 'प्रदेश:B-ORG', 'सरकार:I-ORG', 'जनसामान्य:O', 'को:O', 'आवश्यक:O', 'वस्तुएं:O', 'उचित:O', 'मूल्य:O', 'एवं:O', 'निर्धारित:O', 'मात्रा:O', 'में:O', 'उपलब्ध:O', 'कराने:O', 'के:O', 'लिए:O', 'प्रतिबद्घ:O', 'है:O', 'और:O', 'इसके:O', 'लिए:O', 'समय:O', '-:O', 'समय:O', 'पर:O', 'निर्देश:O', 'जारी:O', 'किए:O', 'जाते:O', 'हैं:O', 'बावजूद:O', 'इसके:O', 'हाल:O', 'के:O', 'दिनों:O', 'में:O', 'कई:O', 'जिंसों:O', 'की:O', 'कीमतों:O', 'में:O', 'तेजी:O', 'इजाफा:O', 'हुआ:O', 'है:O', '।:O'],
    ['इसके:O', 'बाद:O', 'श्रीलंका:B-LOC', 'के:O', 'खिलाफ:O', '264:B-MISC', 'रन:O', 'तब:O', 'बनाए:O', 'थे:O', ',:O', 'जब:O', 'मैं:O', 'उंगली:O', 'में:O', 'चोट:O', 'लगने:O', 'की:O', 'वजह:O', 'से:O', 'तीन:O', 'महीने:O', 'बाद:O', 'वापसी:O', 'कर:O', 'रहा:O', 'था:O', 'और:O', 'नर्वस:O', 'था:O', 'कि:O', 'मैं:O', 'रन:O', 'बना:O', 'पाऊंगा:O', 'या:O', 'नहीं:O', 'और:O', 'अब:O', '208*:B-MISC', 'तब:O', 'बनाए:O', ',:O', 'जब:O', 'सीरीज:O', 'का:O', 'पहला:B-MISC', 'मैच:O', 'गंवाने:O', 'के:O', 'बाद:O', 'दूसरे:B-MISC', 'मैच:O', 'में:O', 'वापसी:O', 'करना:O', 'जरूरी:O', 'था:O', '।:O'],
    ['भारत:B-LOC', 'का:O', 'मीडिया:O', 'टीवी:O', 'चैनल:O', 'या:O', 'अखबार:O', 'भारत-इज़राइल:O', 'रिश्ते:O', 'पर:O', 'विस्तृत:O', 'जानकारी:O', 'नहीं:O', 'देते:O', 'हैं:O', 'लेकिन:O', 'इज़राइल:B-LOC', 'की:O', 'अखबारों:O', 'से:O', 'पता:O', 'चलता:O', 'है:O', 'कि:O', 'भारत-इज़राइल:O', 'रिश्ता:O', 'पिछले:O', 'कुछ:O', 'सालों:O', 'में:O', 'कितना:O', 'प्रगाढ़:O', 'और:O', 'गहरा:O', 'हो:O', 'गया:O', 'है:O', '.:O'],
    ['सुनवाई:O', 'के:O', 'दौरान:O', 'कार्यकर्ता:O', 'ने:O', 'कहा:O', 'कि:O', 'उन्हें:O', 'अपनी:O', 'जिंदगी:O', 'से:O', 'प्यार:O', 'है:O', 'और:O', 'वह:O', 'अफ्सपा:O', 'हटाने:O', 'की:O', 'मांग:O', 'को:O', 'लेकर:O', 'अनशन:O', 'को:O', 'हथियार:O', 'के:O', 'तौर:O', 'पर:O', 'इस्तेमाल:O', 'कर:O', 'रही:O', 'हैं:O', 'क्योंकि:O', 'इसका:O', '‘ज्यादा:O', 'असर:O', 'होगा:O', '।:O', 'उनका:O', 'कहना:O', 'है:O', 'कि:O', 'यह:O', 'अपराध:O', 'नहीं:O', 'है:O', '.:O'],
    ['दोनों:O', 'प्रधानमंत्रियों:O', 'ने:O', 'सिविल:O', 'न्यूक्लियर:O', 'एनर्जी:O', 'के:O', 'क्षेत्र:O', 'में:O', 'खासकर:O', 'तकनीकी:O', 'प्रशिक्षण:O', 'की:O', 'दिशा:O', 'काम:O', 'करने:O', 'पर:O', 'सहमति:O', 'जताई:O', 'है:O', '.:O'],
    ['दोनों:O', 'बदमाश:O', 'शामली:B-LOC', 'के:O', 'लोहारी:B-LOC', 'जलालाबाद:I-LOC', 'के:O', 'रहने:O', 'वाले:O', 'थे:O', 'और:O', 'दोनों:O', 'पर:O', '50-50:O', 'हजार:B-MISC', 'रुपए:O', 'का:O', 'इनाम:O', 'था:O', '।:O'],
    ['बाद:O', 'में:O', 'दोनों:B-MISC', 'समुदाय:O', 'की:O', 'तरफ:O', 'से:O', 'हिंसा:O', 'भड़क:O', 'उठी:O', '।:O'],
    ['30:B-MISC', 'फीसदी:O', 'से:O', 'कम:O', 'अंक:O', 'लाने:O', 'वालों:O', 'को:O', 'असफल:O', 'घोषित:O', 'कर:O', 'दिया:O', 'जाएगा:O', '.:O'],
    ['ट्रेलर:O', 'में:O', 'दर्शाया:O', 'गया:O', 'है:O', 'कि:O', 'कोई:O', 'इसे:O', 'मारने:O', 'के:O', 'लिए:O', 'सात:B-MISC', 'दिनों:O', 'का:O', 'समय:O', 'तय:O', 'करता:O', 'है:O', '।:O'],
    ['ऐसे:O', 'में:O', 'लोगों:O', 'को:O', 'इलाज:O', 'के:O', 'लिये:O', 'बल्लभगढ़:I-LOC', 'व:O', 'फरीदाबाद:I-LOC', 'जाना:O', 'पड़ता:O', 'है:O', '।:O'],
    ['आल्पस:B-ORG', 'कंपनी:O', 'के:O', 'बाहर:O', 'धरने:O', 'पर:O', 'बैठे:O', '14०:B-MISC', 'कर्मचारियों:O', 'ने:O', 'अपनी:O', 'मांगों:O', 'को:O', 'लेकर:O', 'जमकर:O', 'नारेबाजी:O', 'कर:O', 'धरना:O', 'प्रदर्शन:O', 'किया:O', '।:O'],
    ['बजाज:B-PER', 'के:O', 'सवाल:O', 'को:O', 'सुनने:O', 'के:O', 'बाद:O', 'मुझे:O', 'यकीन:O', 'है:O', 'कि:O', 'कोई:O', 'भी:O', 'इस:O', 'दावे:O', 'को:O', 'नहीं:O', 'मानेगा:O', 'कि:O', 'लोग:O', 'डरे:O', 'हुए:O', 'हैं:O', '।:O'],
    ['फिर:O', 'हीरो-हीरोइन:O', 'हमेशा:O', 'तो:O', 'साथ:O', 'रह:O', 'नहीं:O', 'सकते:O', 'तो:O', 'एकविरह:O', 'गीत:O', 'तो:O', 'गाना:O', 'ही:O', 'पड़ेगा:O', 'हीरो:O', 'को:O', '(:O', 'याद:O', 'आ:O', 'रही:O', 'है:O', 'टाइप:O', '):O', '।:O'],
    ['जानकारी:O', 'के:O', 'अनुसार:O', 'क्षेत्रीय:O', 'अस्पताल:O', 'मंे:O', 'इस:O', 'समय:O', 'विशेषज्ञ:O', 'चिकित्सकों:O', 'के:O', '25:B-MISC', 'पद:O', 'सृजित:O', 'किए:O', 'गए:O', 'हैं:O', 'जिनमें:O', 'से:O', '17:B-MISC', 'पद:O', 'ही:O', 'भरे:O', 'हैं:O', ',:O', 'एक:B-MISC', 'डाक्टर:O', 'आईजीएमसी:B-MISC', 'मंे:O', 'डेपुटेशन:O', 'पर:O', 'है:O', ',:O', 'जबकि:O', 'शेष:O', 'खाली:O', 'चल:O', 'रहे:O', 'हैं:O', '।:O'],
    ['साइबर:O', 'क्षेत्र:O', 'में:O', 'घुसपैठ:O', 'करके:O', 'हमारे:O', 'संचार:O', 'व:O', 'सूचना:O', 'तंत्र:O', 'में:O', 'सेंध:O', 'लगा:O', 'रहा:O', 'है:O', '।:O'],
    ['हरियाणा:B-LOC', 'के:O', 'वित्त:B-MISC', 'एवं:O', 'राजस्व:B-MISC', 'मंत्री:I-MISC', 'कैप्टन:B-MISC', 'अभिमन्यु:B-PER', 'ने:O', 'कहा:O', 'कि:O', 'खाप:O', 'व्यवस्था:O', 'भारत:B-LOC', 'की:O', 'मूल:O', 'शासन:O', 'व्यवस्था:O', 'है:O', 'पढ़ें:O'],
    ['अपनी:O', 'इस:O', 'हालत:O', 'के:O', 'लिए:O', 'महिला:O', 'कैदी:O', 'ने:O', 'जेल:O', 'प्रबंधन:O', 'को:O', 'जिम्मेदार:O', 'ठहराया:O', '.:O'],
    ['यह:O', 'तो:O', 'जगज़ाहिर:O', 'है:O', 'कि:O', 'खेल:O', 'और:O', 'विवाद:O', 'दोनों:O', 'का:O', 'आपस:O', 'में:O', 'चोली-दामन:O', 'का:O', 'साथ:O', 'है:O', '.:O'],
    ['इसके:O', 'बादवाले:O', 'एक:O', 'माह:O', 'तक:O', 'कुछ:O', 'पका:O', 'हुआ:O', 'आहार:O', 'दिया:O', 'जा:O', 'सकता:O', 'है:O', '।:O'],
    ['हालांकि:O', 'रक्षा:O', 'मंत्रालय:O', 'में:O', 'उनका:O', 'कार्यकाल:O', 'आसान:O', 'नहीं:O', 'था:O', 'क्योंकि:O', 'वह:O', 'एक:O', 'ऐसे:O', 'नेता:O', 'के:O', 'रूप:O', 'में:O', 'सफल:O', 'रहीं:O', 'जिन्होंने:O', 'उद्योग:O', 'और:O', 'सशस्त्र:O', 'बलों:O', 'दोनों:O', 'का:O', 'विश्वास:O', 'जीता:O', 'था:O', '.:O']
]



### CALCULATING CLASSWISE PRECISION,RECALL AND F1 SCORE AND ALSO MACRO F1 SCORE

In [71]:

def calculate_metrics(tokenizer_outputs, ground_truths):
  tp = len(set(tokenizer_outputs ) & set(ground_truths ))
  fp = len(set(tokenizer_outputs ) - set(ground_truths ))
  fn = len(set(ground_truths) - set(tokenizer_outputs))
  precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
  recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
  if precision + recall > 0:
      f_score = 2 * (precision * recall) / (precision + recall)
  else:
      f_score = 0.0
  return [precision, recall, f_score]



def get_all_tokens(list,class_name):
    l=[]
    for innerlist in list:
        for value in innerlist:
            if class_name==value.split(':')[1]:
                l.append(value.split(':')[0])
    return l

def get_all_tokens_from_num(listt,class_name):
    l=[]
    for innerlist in listt:
        for value in innerlist:
            if (class_name==value.split(':')[1]):
                l.append(value.split(':')[0])
            
    return l

class_names=['B-PER','I-PER','B-LOC','I-LOC','B-ORG','I-ORG','B-MISC','I-MISC','O']

Q1_Q2=[]
for classs in class_names:
    Q1_Q2.append(calculate_metrics(get_all_tokens(pred_outputs,classs),get_all_tokens(Q1_List,classs)))

macro_f1_score=0.0
for i in range(len(class_names)):
    print("Precision of ",class_names[i],":",Q1_Q2[i][0])
    print("Recall of ",class_names[i],":",Q1_Q2[i][1])
    print("f1_score of ",class_names[i],":",Q1_Q2[i][2])
    print()
    macro_f1_score=macro_f1_score+Q1_Q2[i][2]

macro_f1_score/=9
print("Macro F1 Score :",macro_f1_score)

Precision of  B-PER : 0.3333333333333333
Recall of  B-PER : 0.5
f1_score of  B-PER : 0.4

Precision of  I-PER : 0.0
Recall of  I-PER : 0.0
f1_score of  I-PER : 0.0

Precision of  B-LOC : 0.5
Recall of  B-LOC : 0.7142857142857143
f1_score of  B-LOC : 0.588235294117647

Precision of  I-LOC : 0.0
Recall of  I-LOC : 0.0
f1_score of  I-LOC : 0.0

Precision of  B-ORG : 0.125
Recall of  B-ORG : 0.3333333333333333
f1_score of  B-ORG : 0.18181818181818182

Precision of  I-ORG : 0.0
Recall of  I-ORG : 0.0
f1_score of  I-ORG : 0.0

Precision of  B-MISC : 0.0
Recall of  B-MISC : 0.0
f1_score of  B-MISC : 0.0

Precision of  I-MISC : 0.0
Recall of  I-MISC : 0.0
f1_score of  I-MISC : 0.0

Precision of  O : 0.8588588588588588
Recall of  O : 0.9079365079365079
f1_score of  O : 0.8827160493827161

Macro F1 Score : 0.22808550281317164


In [46]:
# ! rm -rf tokenizer